# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [58]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [60]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [61]:
ed = Website("https://amazon.com")
ed.links
# print(ed.get_contents())

['https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088',
 'https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [62]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [63]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [65]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [66]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://amazon.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088
https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496


In [67]:
def get_links(url):
    website = Website(url)
    MODEL = "llama3.2"
    openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [68]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://amazon.com")
huggingface.links

['https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088',
 'https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496']

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)}
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://amazon.com"))

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [71]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [72]:
get_brochure_user_prompt("Amazon", "https://amazon.com")

Found links: {'links': [{'type': 'contact us page', 'url': 'https://www.amazon.com/service-customer-service'}, {'type': 'employees/careers page', 'url': 'https://www.amazon.jobs/'}]}


"You are looking at a company called: Amazon\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nAmazon.com\nWebpage Contents:\nEnter the characters you see below\nSorry, we just need to make sure you're not a robot. For best results, please make sure your browser is accepting cookies.\nType the characters you see in this image:\nTry different image\nContinue shopping\nConditions of Use\nPrivacy Policy\n© 1996-2014, Amazon.com, Inc. or its affiliates\n\n\n\ncontact us page\nWebpage Title:\nAmazon.com Page Not Found\nWebpage Contents:\n\n\n\n\nemployees/careers page\nWebpage Title:\nAmazon.jobs: Help us build Earth’s most customer-centric company.\nWebpage Contents:\nSkip to main content\nHome\nTeams\nLocations\nJob categories\nMy career\nMy applications\nMy profile\nAccount security\nSettings\nSign out\nResources\nDisability accommodations\nBenefits\nInclusive exp

In [73]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [74]:
create_brochure("Amazon", "https://amazon.com")

Found links: {'links': [{'type': 'careers page', 'url': 'https://ats.amazon.com/ats/jobs/'}]}


# Amazon Brochure

[Cover Page: A logo of a smiley monkey with the Amazon logo]

## Welcome to Amazon

At Amazon, we're passionate about making it easier and more enjoyable for everyone to shop, live, and work. Our mission is to be Earth's most customer-centric company, where customers can find and discover anything they might want to buy online.

### Company Culture

We value our employees as much as our customers. We believe that diversity, inclusion, and respect are essential building blocks of a great team. We strive to create a workplace where everyone feels welcome and empowered to do their best work.

At Amazon, you can find like-minded people who share your values and passions. Our company culture is built on three core principles:

* **Customer Obsession**: We're dedicated to delivering the highest level of customer service.
* **Owner's Mindset**: We think like entrepreneurs and owners, always looking for ways to improve and innovate.
* **Invent and Simplify**: We strive to create a work environment that encourages creativity, collaboration, and simplicity.

### Customers

We're proud to serve millions of customers across the globe. At Amazon, we believe in providing convenient, fast, and reliable shipping options to meet our customers' needs.

**Key Customer Services**

* Free Same-Day Delivery on eligible items
* Prime Membership benefits (including free two-day shipping, streaming of movies and TV shows, and music streaming)
* Easy returns and exchanges

### Careers at Amazon

We're looking for talented individuals to join our team. Whether you're a software engineer, customer service representative, or delivery driver, you play a critical role in delivering exceptional experiences for our customers.

**Benefits of Working at Amazon**

* Competitive salaries and bonuses
* Comprehensive benefits package (including health insurance, retirement savings plan, and flexible paid time off)
* Opportunities for growth and career development
* Collaborative and supportive work environment

If you're passionate about making a difference through innovative technology, customer-centric service, or operational excellence, apply to join our team today!

[Back Cover: A logo of the Amazon monkey with a bright future ahead]

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [85]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [86]:
stream_brochure("Amazon", "https://amazon.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.amazon.com/companyinformation'}, {'type': 'help center', 'url': 'https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088'}, {'type': 'privacy policy', 'url': 'https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496'}]}


**Welcome to Amazon Brochure**
=====================================

**Mission & Vision**
-------------------

Amazon is a pioneer in e-commerce, with a mission to be Earth's most customer-centric company, where customers can find and discover anything they might want to buy online. Our vision is to inspire and reward customers to unlock their full potential.

**Company Culture**
------------------

* **Innovative Spirit**: We encourage creativity, experimentation, and learning.
* **Customer Obsession**: We put our customers at the heart of everything we do.
* **Strong Work Ethic**: We are committed to having a positive impact on our employees and our communities.
* **Long-term Thinking**: We think about the future and make long-term commitments.

**Customer Relationships**
------------------------

At Amazon, we pride ourselves on building strong relationships with our customers. With over 300 million active customers worldwide, you can find anything you need on our platform, from everyday essentials to rare and hard-to-find items. Our customer satisfaction ratings are among the highest in e-commerce, ensuring that your experience with us is always excellent.

**Careers & Jobs**
-----------------

Join our team of innovators! At Amazon, we offer a wide range of exciting career opportunities, including:

* **Software Engineers**: Help develop the technology that powers Amazon's platform.
* **Customer Service Representatives**: Provide exceptional support to our customers.
* **Operations Professionals**: Ensure the smooth operation of our logistics and delivery network.

**Sustainability & Social Responsibility**
----------------------------------------

At Amazon, we are committed to a sustainable future for all. We have set ambitious sustainability goals, including:

* **Renewable Energy**: 50% of our energy comes from renewable sources.
* **Carbon Neutral**: Our operations are carbon neutral.

**Get in Touch**
-----------------

Ready to shop with us or learn more about our company? Visit [www.amazon.com](http://www.amazon.com) today!

**Contact Information**

Address: Amazon HQ, 410 Terry Avenue North, Seattle, WA 98109-5210

Phone: +1 (206) 622-2335

Email: [customer.service@amazon.com](mailto/customer.service@amazon.com)

Note: The provided pages do not contain any relevant information about the company's products or services. However, this brochure provides a brief overview of Amazon's mission, culture, customer relationships, career opportunities, sustainability initiatives, and contact information.

In [ ]:
 

stream_brochure("Amazon","https://amazon.com")

<table  >
   
 
      
          
 In this exercise we learnt how to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
       
  
</table>